# 01 — Data Exploration & Quality Assessment

**Objective**: Understand TravelTide's customer journey, data quality, and behavioral patterns.

**Key Questions**:
- What does the customer journey look like?
- What are the data quality issues?
- What behavioral patterns exist in sessions and bookings?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Load raw data
data_dir = Path('../data')
users = pd.read_csv(data_dir / 'users_full.csv')
sessions = pd.read_csv(data_dir / 'sessions_full.csv')
flights = pd.read_csv(data_dir / 'flights_full.csv')
hotels = pd.read_csv(data_dir / 'hotels_full.csv')

print(f"Users: {len(users):,} rows")
print(f"Sessions: {len(sessions):,} rows")
print(f"Flights: {len(flights):,} rows")
print(f"Hotels: {len(hotels):,} rows")

## Data Quality Assessment

In [ ]:
# Missing values analysis
def analyze_missing(df, name):
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    result = pd.DataFrame({
        'Missing': missing,
        'Percentage': missing_pct
    }).sort_values('Missing', ascending=False)
    print(f"\n{name} - Missing Values:")
    print(result[result['Missing'] > 0])
    return result

users_missing = analyze_missing(users, 'Users')
sessions_missing = analyze_missing(sessions, 'Sessions')
flights_missing = analyze_missing(flights, 'Flights')
hotels_missing = analyze_missing(hotels, 'Hotels')

In [ ]:
# Join sessions with users for behavioral analysis
sessions['session_start'] = pd.to_datetime(sessions['session_start'])
sessions['session_end'] = pd.to_datetime(sessions['session_end'])
sessions['duration_minutes'] = (sessions['session_end'] - sessions['session_start']).dt.total_seconds() / 60

# Merge with users
session_user = sessions.merge(users, on='user_id', how='left')

print(f"Session-User merged data: {len(session_user):,} rows")
print(f"Unique users in sessions: {session_user['user_id'].nunique():,}")

## Behavioral Patterns

In [ ]:
# Session behavior analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Session duration distribution
axes[0,0].hist(session_user['duration_minutes'].dropna(), bins=50, alpha=0.7)
axes[0,0].set_title('Session Duration Distribution')
axes[0,0].set_xlabel('Duration (minutes)')

# Page clicks distribution
axes[0,1].hist(session_user['page_clicks'].dropna(), bins=30, alpha=0.7)
axes[0,1].set_title('Page Clicks Distribution')
axes[0,1].set_xlabel('Page Clicks')

# Booking rates
booking_rates = pd.DataFrame({
    'Flight Bookings': [session_user['flight_booked'].mean()],
    'Hotel Bookings': [session_user['hotel_booked'].mean()],
    'Cancellations': [session_user['cancellation'].mean()]
})
booking_rates.T.plot(kind='bar', ax=axes[1,0], legend=False)
axes[1,0].set_title('Booking & Cancellation Rates')
axes[1,0].set_ylabel('Rate')

# Sessions per user
sessions_per_user = session_user.groupby('user_id').size()
axes[1,1].hist(sessions_per_user, bins=20, alpha=0.7)
axes[1,1].set_title('Sessions per User')
axes[1,1].set_xlabel('Number of Sessions')

plt.tight_layout()
plt.show()

## Customer Demographics

In [ ]:
# Demographics analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 8))

# Gender distribution
users['gender'].value_counts().plot(kind='pie', ax=axes[0,0], autopct='%1.1f%%')
axes[0,0].set_title('Gender Distribution')

# Marital status
users['married'].value_counts().plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Marital Status')
axes[0,1].tick_params(axis='x', rotation=0)

# Children
users['has_children'].value_counts().plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Has Children')
axes[1,0].tick_params(axis='x', rotation=0)

# Top countries
users['home_country'].value_counts().head(10).plot(kind='barh', ax=axes[1,1])
axes[1,1].set_title('Top 10 Countries')

plt.tight_layout()
plt.show()

## Booking Behavior Analysis

In [ ]:
# Flight booking analysis
if len(flights) > 0:
    flights_with_sessions = flights.merge(sessions[['trip_id', 'user_id']], on='trip_id', how='left')

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Base fare distribution
    axes[0].hist(flights['base_fare_usd'].dropna(), bins=50, alpha=0.7)
    axes[0].set_title('Flight Base Fare Distribution')
    axes[0].set_xlabel('Base Fare (USD)')

    # Seats distribution
    axes[1].hist(flights['seats'].dropna(), bins=10, alpha=0.7)
    axes[1].set_title('Seats per Booking')
    axes[1].set_xlabel('Number of Seats')

    # Return flight rate
    return_rate = flights['return_flight_booked'].mean()
    axes[2].bar(['One-way', 'Return'], [1-return_rate, return_rate])
    axes[2].set_title('Flight Type Distribution')
    axes[2].set_ylabel('Proportion')

    plt.tight_layout()
    plt.show()

# Hotel booking analysis
if len(hotels) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Hotel price distribution
    axes[0].hist(hotels['hotel_per_room_usd'].dropna(), bins=50, alpha=0.7)
    axes[0].set_title('Hotel Price per Room Distribution')
    axes[0].set_xlabel('Price per Room (USD)')

    # Nights distribution
    axes[1].hist(hotels['nights'].dropna(), bins=15, alpha=0.7)
    axes[1].set_title('Nights per Booking')
    axes[1].set_xlabel('Number of Nights')

    # Rooms distribution
    axes[2].hist(hotels['rooms'].dropna(), bins=10, alpha=0.7)
    axes[2].set_title('Rooms per Booking')
    axes[2].set_xlabel('Number of Rooms')

    plt.tight_layout()
    plt.show()

## Key Insights Summary

In [ ]:
# Generate summary statistics
print("=== KEY INSIGHTS ===")
print(f"\n📊 Data Scale:")
print(f"  • {users['user_id'].nunique():,} unique users")
print(f"  • {len(sessions):,} total sessions")
print(f"  • {sessions_per_user.mean():.1f} avg sessions per user")

print(f"\n🎯 Conversion Rates:")
print(f"  • Flight booking rate: {session_user['flight_booked'].mean():.1%}")
print(f"  • Hotel booking rate: {session_user['hotel_booked'].mean():.1%}")
print(f"  • Cancellation rate: {session_user['cancellation'].mean():.1%}")

print(f"\n💰 Revenue Metrics:")
if len(flights) > 0:
    print(f"  • Avg flight fare: ${flights['base_fare_usd'].mean():.0f}")
if len(hotels) > 0:
    print(f"  • Avg hotel price: ${hotels['hotel_per_room_usd'].mean():.0f}/room")

print(f"\n⚠️  Data Quality Issues:")
for df_name, missing_df in [('Users', users_missing), ('Sessions', sessions_missing)]:
    high_missing = missing_df[missing_df['Percentage'] > 10]
    if len(high_missing) > 0:
        print(f"  • {df_name}: {len(high_missing)} columns with >10% missing")